In [1]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("MiAplicacionSpark") \
    .getOrCreate()

11) ¿Cuánto se debió haber facturado por las ventas de los productos SKUB06?

In [40]:
df2 = spark.read.csv(r'pedidos.csv',  header=True, 
                     inferSchema=True,
                     sep=';',
                     encoding='latin1')

In [41]:
df2.show()

+-----------------------------+------+----------------------+------------------+--------+------------------+--------------+-------------+--------------+----------------+---------+------------+-----------+
|Número de Pedido - Tip Compra|   SKU|Categoría de Descuento|   Precio de Venta|Unidades|Costo del Producto|Costo de Envio|Costo Empaque|Fecha de Envio|Fecha de Llegada|     País|      Ciudad|Codigo País|
+-----------------------------+------+----------------------+------------------+--------+------------------+--------------+-------------+--------------+----------------+---------+------------+-----------+
|               98094262Normal|CB01-5|                   Non| 666.0899999999999|       3|                 8|             6|            3|         36508|           36527|VENezuela|     Caracas|         VE|
|               98094261Normal|CC01-2|                   Non|            105.06|       2|              8,55|          4,55|            2|         36509|           36529| PARaguay| 

In [52]:
from pyspark.sql.functions import regexp_replace

df2 = df2.withColumn('Costo de Envio', regexp_replace('Costo de Envio', ',', '.'))
df2 = df2.withColumn('Costo del Producto', regexp_replace('Costo del Producto', ',', '.'))

In [53]:
df2.createOrReplaceTempView("venta")
df2.printSchema()


root
 |-- Número de Pedido - Tip Compra: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Categoría de Descuento: string (nullable = true)
 |-- Precio de Venta: double (nullable = true)
 |-- Unidades: integer (nullable = true)
 |-- Costo del Producto: string (nullable = true)
 |-- Costo de Envio: string (nullable = true)
 |-- Costo Empaque: integer (nullable = true)
 |-- Fecha de Envio: integer (nullable = true)
 |-- Fecha de Llegada: integer (nullable = true)
 |-- País: string (nullable = true)
 |-- Ciudad: string (nullable = true)
 |-- Codigo País: string (nullable = true)



In [54]:
result = spark.sql("""
                SELECT
                SUM(`Precio de venta` * `Unidades`) as total
                FROM venta
                WHERE SKU LIKE 'B06%' """)
result.show()



+------------------+
|             total|
+------------------+
|104711.03000000033|
+------------------+



12) ¿Hay algún pedido que tenga utilidad negativa? *

In [64]:
result = spark.sql("""
                SELECT
                `Precio de Venta`- (`Costo del Producto`+`Costo de Envio`+`Costo Empaque`)
                from venta
                order by 1
                 """)
result.show()

+---------------------------------------------------------------------------+
|(Precio de Venta - ((Costo del Producto + Costo de Envio) + Costo Empaque))|
+---------------------------------------------------------------------------+
|                                                        -0.8520000000000003|
|                                                       -0.41499999999999915|
|                                                        -0.2920000000000016|
|                                                        0.36799999999999855|
|                                                         0.3879999999999981|
|                                                         0.9279999999999973|
|                                                                     1.1875|
|                                                         1.3679999999999986|
|                                                         1.3679999999999986|
|                                                         1.4679

13) Calcule la cantidad de unidades promedio de los pedidos cuya categoría de
descuento fue Cyber Monda

In [65]:
result = spark.sql("""
                SELECT
                avg(`Unidades`)
                from venta
                where `Categoría de Descuento` = 'Cyber Monday'
    
                 """)
result.show()

+------------------+
|     avg(Unidades)|
+------------------+
|1.9745738225946259|
+------------------+



14) ¿Cuántos pedidos se realizaron en Argentina? *

In [68]:
spark.sql("""
          SELECT DISTINCT count(*) FROM `venta` as v
          WHERE v.`Codigo País` = "AR";
          """).show()

+--------+
|count(1)|
+--------+
|    2848|
+--------+

